In [123]:
%reload_ext autoreload
%autoreload 2

import os
import json
import glob
# from semproc.parser import Parser
from semproc.identifier import Identify
from semproc.rawresponse import RawResponse

In [6]:
def _prep(filepath):
    with open(filepath, 'r') as f:
        response = f.read()

    response = response.replace('\\\n', '').replace('\r\n', '').replace('\\r', '').replace('\\n', '').replace('\n', '')
    return response.decode('utf-8', errors='replace').encode('unicode_escape')

In [125]:
files = [
    ('http://cwic.nasa.gov/something?service=csw&version=2.0.2&request=getcapabilities', 
     '../response_examples/cwic_csw_v2_0_2.xml'),
    ('http://www.data.gov?service=csw&version=2.0.2&request=getrecords', 
     '../response_examples/datagov_csw_202_getrecords_iso.xml'),
    ('http://www.data.gov?service=csw&version=2.0.2&request=getrecords', 
     '../response_examples/datagov_csw_202_getrecords_dif.xml'),
    ('http://www.data.gov?service=csw&version=2.0.2&request=getrecords', 
     '../response_examples/datagov_csw_202_getrecords_fgdc.xml'),
    ('http://www.data.gov/this/is/rdf', 
     '../response_examples/datagov_9bcffa1c-6164-4635-bc2c-6c98cce59d7b.rdf'),
    ('http://www.data.gov/iso/ds', 
     '../response_examples/iso-19115_ds.xml'),
    ('http://www.data.gov/iso/md', 
     '../response_examples/iso-19115_md.xml'),
    ('http://www.data.gov/iso/mi', 
     '../response_examples/iso-19115_mi.xml'),
    ('http://www.data.gov/iso/srv', 
     '../response_examples/iso-19119_as_md.xml'),
    ('http://www.example.gov/oai', 
     '../response_examples/oaipmh_listrecords.xml'),
    ('http://www.example.gov/oai', 
     '../response_examples/oaipmh_identify.xml'),
    ('http://www.example.gov/opendap/wsdl', 
     '../response_examples/opendap_wsdl.xml'),
    ('http://www.example.gov/os/osdd.xml', 
     '../response_examples/opensearch_blended_parameters.xml'),
    ('http://www.example.gov/os/atom', 
     '../response_examples/opensearch_dspace_search_atom.xml'),
    ('http://www.example.gov/or/rss', 
     '../response_examples/opensearch_ngdc_search_rss.xml'),
    ('http://www.example.gov/sos?service=sos&version=1.0.0&request=getcapabilities', 
     '../response_examples/sos_v1_0_0.xml'),
    ('http://www.example.gov/thredds/eol', 
     '../response_examples/thredds_catalog_eol.xml'),
    ('http://www.example.gov/thredds/catalog', 
     '../response_examples/thredds_catalog.xml'),
    ('http://www.data.gov?service=wcs&version=1.0.0&request=describecoverage', 
     '../response_examples/wcs_v1_0_0_describe_coverage.xml'),
    ('http://www.data.gov?service=wcs&version=1.0.0&request=getcapabilities', 
     '../response_examples/wcs_v1_0_0.xml'),
    ('http://www.data.gov?service=wcs&version=1.1.0&request=getcapabilities', 
     '../response_examples/wcs_v1_1_0.xml'),
    ('http://www.data.gov?service=wcs&version=1.1.2&request=getcapabilities', 
     '../response_examples/wcs_v1_1_2.xml'),
    ('http://www.data.gov?service=wfs&version=1.0.0&request=getcapabilities', 
     '../response_examples/wfs_v1_0_0.xml'),
    ('http://www.data.gov?service=wfs&version=1.1.0&request=getcapabilities', 
     '../response_examples/wfs_v1_1_0.xml'),
    ('http://www.data.gov?service=wms&version=1.3.0&request=getcapabilities', 
     '../response_examples/wms_v1_3_0.xml'),
    ('http://www.data.gov?service=wms&version=1.3.0&request=getcapabilities', 
     '../response_examples/wms_v1.3.0.xml')
]

In [126]:
yamls = glob.glob('../semproc/configs/*_identifier.yaml')

for url, filepath in files:
    response = _prep(filepath)
    
    i = Identify(yamls, response, url)
    matches = i.identify()
    
    print
    print 'Identification----------------'
    print '\t', url, filepath
    print '\t', matches
    


Identification----------------
	http://cwic.nasa.gov/something?service=csw&version=2.0.2&request=getcapabilities ../response_examples/cwic_csw_v2_0_2.xml
	[{'protocol': 'OGC', 'service': {'name': 'CSW', 'request': 'GetCapabilities'}}]

Identification----------------
	http://www.data.gov?service=csw&version=2.0.2&request=getrecords ../response_examples/datagov_csw_202_getrecords_iso.xml
	[{'resultset': {'dialect': ['http://www.isotc211.org/2005/gmd'], 'name': 'CSW', 'request': 'GetRecords'}, 'protocol': 'OGC'}]

Identification----------------
	http://www.data.gov?service=csw&version=2.0.2&request=getrecords ../response_examples/datagov_csw_202_getrecords_dif.xml
	[{'resultset': {'dialect': ['http://gcmd.gsfc.nasa.gov/Aboutus/xml/dif/'], 'name': 'CSW', 'request': 'GetRecords'}, 'protocol': 'OGC'}]

Identification----------------
	http://www.data.gov?service=csw&version=2.0.2&request=getrecords ../response_examples/datagov_csw_202_getrecords_fgdc.xml
	[{'resultset': {'dialect': ['http://

In [115]:
import yaml
import re
# y = yaml.load("""
# - name: SOS
#   request: GetCapabilities
#   filters:
#     ands:

#       - type: simple
#         object: content
#         value: 'http://www.opengis.net/sos'
#       - type: simple
#         object: content
#         value: 'Operation name="DescribeSensor"'

#       - type: complex
#         operator: ors
#         filters:
#           - type: simple
#             object: url
#             value: 'REQUEST=GETCAPABILITIES'
#           - type: simple
#             object: content
#             value: '<GetCapabilities>'
#           - type: simple
#             object: content
#             value: 'name="GetCapabilities"'""")

with open('../semproc/configs/test_ogc.yaml', 'r') as f:
    y = yaml.load(f.read())


response = _prep('../response_examples/cwic_csw_v2_0_2.xml')
url = 'http://cwic.nasa.gov/something?service=csw&version=2.0.2&request=getcapabilities'
xml = etree.fromstring(response)

def _filter(xml, source_content, source_url, operator, filters, clauses):
    '''
    generate a list of dicts for operator and booleans
    that can be rolled up into some bool for a match
    '''
    for f in filters:
        filter_type = f['type']

        if filter_type == 'complex':
            filter_operator = f['operator']
            clauses.append(_filter(xml, source_content, source_url, filter_operator, f['filters'], []))
            print 'COMPLEX', filter_operator
        elif filter_type == 'simple':
            filter_object = source_content if f['object'] == 'content' else source_url
            filter_value = f['value']

            # TODO: a better solution than this
            filter_value = filter_value.upper()
            filter_object = filter_object.upper()

            clauses.append(filter_value in filter_object)
            
            print 'SIMPLE', f['value'], filter_value in filter_object
        elif filter_type == 'regex':
            filter_object = source_content if f['object'] == 'content' else source_url
            filter_value = f['value']
            clauses.append(len(re.findall(filter_value, filter_object)) > 0)
            
            print 'regex', f['value'], len(re.findall(filter_value, filter_object)) > 0
        elif filter_type == 'xpath':
            # if the filter is xpath, we can only run against
            # the provided xml (parser) and ONLY evaluate for existence
            # ie the xpath returned some element, list, text value
            # but we don't care what it returned
            xpath = f['value']
            if xml is None:
                # nothing to find, this is an incorrect filter
                clauses.append(False)

            # try the xpath but there could be namespace or
            # other issues (also false negatives!)
            try:
                clause = xml.xpath(xpath) not in [None, '', []]
            except:
                clause = False

            print 'XPATH', f['value'], clause
            clauses.append(clause)

    print 'filtered: ', {operator: clauses}
    return {operator: clauses}

def _evaluate(clauses, sums):
    '''
    evaluate a list a dicts where the key is
    the operator and the value is a list of
    booleans
    '''
    if isinstance(clauses, bool):
        # so this should be the rolled up value
        return clauses

    for k, v in clauses.iteritems():
        if isinstance(v, dict):
            e = _evaluate(v, 0)
            print 'dict', k, e, sums
            return sums + e
        elif isinstance(v, list) and not all(isinstance(i, bool) for i in v):
            # TODO: this is not a good assumption
            print 'i am a list', v
            new_list = []
            for i in v:
                new_list.append(_evaluate(i, 0))
            #return sums
            v = new_list
        if k == 'ands':
            # everything must be true
            sums += sum(v) == len(v)
            print 'ands = ', sum(v) == len(v), v
        elif k == 'ors':
            # any one must be true
            sums += sum(v) > 0
            print 'ors = ', sum(v) > 0, v

    return sums


# so it's not respecting the parent *and* instead 
# running the or and we are getting false matches on the getcapabilities flags.
    



In [116]:
for x in y:
    for k, v in x.iteritems():
        if k == 'name' or v is None:
            continue

        for option in v:
            filters = option['filters']

            for i, j in filters.iteritems():
                #print i, j
                final_sums = _evaluate({i: _filter(xml, response, url, i, j, [])}, 0)
                print '----------------------------'
                print 'returned: ', final_sums
                if final_sums:
                    print 'MATCHED: ', x['name'], option['name']

        

SIMPLE GetRecordsResponse False
SIMPLE http://www.opengis.net/cat/csw/ True
filtered:  {'ands': [False, True]}
ands =  False [False, True]
dict ands 0 0
----------------------------
returned:  0
SIMPLE http://www.opengis.net/wms False
SIMPLE <!DOCTYPE WMT_MS_Capabilities SYSTEM "http://schemas.opengis.net/wms/1.1.1/WMS_MS_Capabilities.dtd" False
filtered:  {'ors': [False, False]}
COMPLEX ors
SIMPLE REQUEST=GETCAPABILITIES True
SIMPLE <GetCapabilities> False
SIMPLE name="GetCapabilities" True
filtered:  {'ors': [True, False, True]}
COMPLEX ors
filtered:  {'ands': [{'ors': [False, False]}, {'ors': [True, False, True]}]}
i am a list [{'ors': [False, False]}, {'ors': [True, False, True]}]
ors =  False [False, False]
ors =  True [True, False, True]
ands =  False [0, 1]
dict ands 0 0
----------------------------
returned:  0
SIMPLE http://www.opengis.net/cat/csw True
SIMPLE SERVICE=CSW True
SIMPLE <csw:Capabilities True
filtered:  {'ors': [True, True]}
COMPLEX ors
SIMPLE REQUEST=GETCAPABILIT

In [10]:
#### testing the ncml parser that i forgot i had
# from semproc.preprocessors.thredds_preprocessors import NcmlReader
from semproc.processor import Processor
from semproc.utils import extract_element_tag
from semproc.utils import generate_short_uuid
from semproc.utils import tidy_dict
from semproc.xml_utils import extract_elems, extract_attrib

ncml = '''<netcdf xmlns="http://www.unidata.ucar.edu/namespaces/netcdf/ncml-2.2" location="/opendap/hyrax/Aqua_AIRS_Level3/AIRX3ST8.005/2007/AIRS.2007.01.02.L3.RetStd008.v5.0.14.0.G07219003002.hdf">
  <dimension name="TempPrsLvls" length="24"/>
  <dimension name="H2OPrsLvls" length="12"/>
  <dimension name="CoarseCldLyrs" length="3"/>
  <dimension name="CO_Layers" length="7"/>
  <dimension name="CH4_Layers" length="3"/>
  <dimension name="EmisLvls" length="4"/>
  <dimension name="TempPrsLvls_1" length="24"/>
  <dimension name="H2OPrsLvls_1" length="12"/>
  <dimension name="CoarseCldLyrs_1" length="3"/>
  <dimension name="CO_Layers_1" length="7"/>
  <dimension name="CH4_Layers_1" length="3"/>
  <dimension name="EmisLvls_1" length="4"/>
  <dimension name="TempPrsLvls_2" length="24"/>
  <dimension name="H2OPrsLvls_2" length="12"/>
  <dimension name="CoarseCldLyrs_2" length="3"/>
  <dimension name="CO_Layers_2" length="7"/>
  <dimension name="CH4_Layers_2" length="3"/>
  <dimension name="MWEmisLvls_2" length="3"/>
  <dimension name="TempPrsLvls_3" length="24"/>
  <dimension name="H2OPrsLvls_3" length="12"/>
  <dimension name="CoarseCldLyrs_3" length="3"/>
  <dimension name="CO_Layers_3" length="7"/>
  <dimension name="CH4_Layers_3" length="3"/>
  <dimension name="MWEmisLvls_3" length="3"/>
  <variable name="TotalCounts_A">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotCldLiqH2O_A">
    <attribute name="standard_name" type="String" value="atmosphere_cloud_liquid_water_content"/>
    <attribute name="long_name" type="String" value="atmosphere_cloud_liquid_water_content"/>
    <attribute name="units" type="String" value="kg/m2"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotCldLiqH2O_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotCldLiqH2O_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotCldLiqH2O_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotH2OVap_A">
    <attribute name="standard_name" type="String" value="atmosphere_water_vapor_content"/>
    <attribute name="long_name" type="String" value="atmosphere_water_vapor_content"/>
    <attribute name="units" type="String" value="kg/m2"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotH2OVap_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotH2OVap_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotH2OVap_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotO3_A">
    <attribute name="standard_name" type="String" value="atmosphere_moles_of_ozone"/>
    <attribute name="long_name" type="String" value="atmosphere_moles_of_ozone"/>
    <attribute name="units" type="String" value="mol"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotO3_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotO3_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotO3_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfAirTemp_A">
    <attribute name="standard_name" type="String" value="surface_temperature"/>
    <attribute name="long_name" type="String" value="surface_temperature"/>
    <attribute name="units" type="String" value="K"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfAirTemp_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="SurfAirTemp_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfAirTemp_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfSkinTemp_A">
    <attribute name="standard_name" type="String" value="sea_surface_skin_temperature"/>
    <attribute name="long_name" type="String" value="sea_surface_skin_temperature"/>
    <attribute name="units" type="String" value="K"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfSkinTemp_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="SurfSkinTemp_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfSkinTemp_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfPres_A">
    <attribute name="standard_name" type="String" value="surface_air_pressure"/>
    <attribute name="long_name" type="String" value="surface_air_pressure"/>
    <attribute name="units" type="String" value="Pa"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfPres_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="SurfPres_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="OLR_A">
    <attribute name="standard_name" type="String" value="toa_outgoing_longwave_flux"/>
    <attribute name="long_name" type="String" value="toa_outgoing_longwave_flux"/>
    <attribute name="units" type="String" value="W/m2"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="OLR_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="OLR_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="ClrOLR_A">
    <attribute name="standard_name" type="String" value="toa_outgoing_longwave_flux_assuming_clear_sky"/>
    <attribute name="long_name" type="String" value="toa_outgoing_longwave_flux_assuming_clear_sky"/>
    <attribute name="units" type="String" value="W/m2"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="ClrOLR_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="ClrOLR_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="EmisIR_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="IREmisLvls_A Latitude Longitude"/>
  </variable>
  <variable name="EmisIR_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="IREmisLvls_A Latitude Longitude"/>
  </variable>
  <variable name="EmisIR_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="IREmisLvls_A Latitude Longitude"/>
  </variable>
  <variable name="EmisIR_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="IREmisLvls_A Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_A">
    <attribute name="standard_name" type="String" value="geopotential_height"/>
    <attribute name="long_name" type="String" value="geopotential_height"/>
    <attribute name="units" type="String" value="m"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="CloudFrc_A">
    <attribute name="standard_name" type="String" value="cloud_area_fraction"/>
    <attribute name="long_name" type="String" value="cloud_area_fraction"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudFrc_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CloudFrc_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudFrc_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudTopPres_A">
    <attribute name="standard_name" type="String" value="air_pressure_at_cloud_top"/>
    <attribute name="long_name" type="String" value="air_pressure_at_cloud_top"/>
    <attribute name="units" type="String" value="Pa"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudTopPres_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CloudTopPres_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudTopPres_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="RelHumid_A">
    <attribute name="standard_name" type="String" value="relative_humidity"/>
    <attribute name="long_name" type="String" value="relative_humidity"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="RelHumid_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="RelHumid_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="H2OVapMMR_A">
    <attribute name="standard_name" type="String" value="humidity_mixing_ratio"/>
    <attribute name="long_name" type="String" value="humidity_mixing_ratio"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="H2OVapMMR_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="H2OVapMMR_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="H2OVapMMR_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="Temperature_A">
    <attribute name="standard_name" type="String" value="air_temperature"/>
    <attribute name="long_name" type="String" value="air_temperature"/>
    <attribute name="units" type="String" value="K"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="Temperature_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="Temperature_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="Temperature_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="CO_total_column_A">
    <attribute name="standard_name" type="String" value="atmosphere_moles_of_carbon_monoxide"/>
    <attribute name="long_name" type="String" value="atmosphere_moles_of_carbon_monoxide"/>
    <attribute name="units" type="String" value="mol"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CO_total_column_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CO_total_column_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CH4_VMR_eff_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CH4_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CH4_VMR_eff_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CH4_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CH4_VMR_eff_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CH4_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CH4_VMR_eff_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CH4_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CO_VMR_eff_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CO_VMR_eff_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CO_VMR_eff_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CO_VMR_eff_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CO_dof_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CO_dof_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CO_dof_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CH4_dof_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CH4_dof_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CH4_dof_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CoarseCloudFrc_A">
    <attribute name="standard_name" type="String" value="cloud_area_fraction_in_atmosphere_layer"/>
    <attribute name="long_name" type="String" value="cloud_area_fraction_in_atmosphere_layer"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudFrc_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudFrc_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudTemp_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudTemp_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudTemp_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudPres_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudPres_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudPres_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="FineCloudFrc_A">
    <attribute name="standard_name" type="String" value="cloud_area_fraction_in_atmosphere_layer"/>
    <attribute name="long_name" type="String" value="cloud_area_fraction_in_atmosphere_layer"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="FineCloudFrc_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="FineCloudFrc_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="CloudTopTemp_A">
    <attribute name="standard_name" type="String" value="air_temperature_at_cloud_top"/>
    <attribute name="long_name" type="String" value="air_temperature_at_cloud_top"/>
    <attribute name="units" type="String" value="K"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudTopTemp_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CloudTopTemp_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudTopTemp_A_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropPres_A">
    <attribute name="standard_name" type="String" value="tropopause_air_pressure"/>
    <attribute name="long_name" type="String" value="tropopause_air_pressure"/>
    <attribute name="units" type="String" value="Pa"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropPres_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TropPres_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropHeight_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropHeight_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TropHeight_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropTemp_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropTemp_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TropTemp_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CO_Verticality_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CO_Verticality_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CO_Verticality_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="RelHumidLiq_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="RelHumidLiq_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="RelHumidLiq_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_A Latitude Longitude"/>
  </variable>
  <variable name="CO_Eff_Press_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CO_Eff_Press_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CO_Eff_Press_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_A Latitude Longitude"/>
  </variable>
  <variable name="CH4_Eff_Press_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="CH4_Eff_Press_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="CH4_Eff_Press_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_A Latitude Longitude"/>
  </variable>
  <variable name="TotalCounts_D">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotCldLiqH2O_D">
    <attribute name="standard_name" type="String" value="atmosphere_cloud_liquid_water_content"/>
    <attribute name="long_name" type="String" value="atmosphere_cloud_liquid_water_content"/>
    <attribute name="units" type="String" value="kg/m2"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotCldLiqH2O_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotCldLiqH2O_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotCldLiqH2O_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotH2OVap_D">
    <attribute name="standard_name" type="String" value="atmosphere_water_vapor_content"/>
    <attribute name="long_name" type="String" value="atmosphere_water_vapor_content"/>
    <attribute name="units" type="String" value="kg/m2"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotH2OVap_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotH2OVap_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotH2OVap_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotO3_D">
    <attribute name="standard_name" type="String" value="atmosphere_moles_of_ozone"/>
    <attribute name="long_name" type="String" value="atmosphere_moles_of_ozone"/>
    <attribute name="units" type="String" value="mol"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotO3_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotO3_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotO3_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfAirTemp_D">
    <attribute name="standard_name" type="String" value="surface_temperature"/>
    <attribute name="long_name" type="String" value="surface_temperature"/>
    <attribute name="units" type="String" value="K"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfAirTemp_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="SurfAirTemp_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfAirTemp_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfSkinTemp_D">
    <attribute name="standard_name" type="String" value="sea_surface_skin_temperature"/>
    <attribute name="long_name" type="String" value="sea_surface_skin_temperature"/>
    <attribute name="units" type="String" value="K"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfSkinTemp_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="SurfSkinTemp_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfSkinTemp_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfPres_D">
    <attribute name="standard_name" type="String" value="surface_air_pressure"/>
    <attribute name="long_name" type="String" value="surface_air_pressure"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="SurfPres_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="SurfPres_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="OLR_D">
    <attribute name="standard_name" type="String" value="toa_outgoing_longwave_flux"/>
    <attribute name="long_name" type="String" value="toa_outgoing_longwave_flux"/>
    <attribute name="units" type="String" value="W/m2"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="OLR_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="OLR_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="ClrOLR_D">
    <attribute name="standard_name" type="String" value="toa_outgoing_longwave_flux_assuming_clear_sky"/>
    <attribute name="long_name" type="String" value="toa_outgoing_longwave_flux_assuming_clear_sky"/>
    <attribute name="units" type="String" value="W/m2"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="ClrOLR_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="ClrOLR_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="EmisIR_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="IREmisLvls_D Latitude Longitude"/>
  </variable>
  <variable name="EmisIR_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="IREmisLvls_D Latitude Longitude"/>
  </variable>
  <variable name="EmisIR_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="IREmisLvls_D Latitude Longitude"/>
  </variable>
  <variable name="EmisIR_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="IREmisLvls_D Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_D">
    <attribute name="standard_name" type="String" value="geopotential_height"/>
    <attribute name="long_name" type="String" value="geopotential_height"/>
    <attribute name="units" type="String" value="m"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="CloudFrc_D">
    <attribute name="standard_name" type="String" value="cloud_area_fraction"/>
    <attribute name="long_name" type="String" value="cloud_area_fraction"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudFrc_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CloudFrc_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudFrc_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudTopPres_D">
    <attribute name="standard_name" type="String" value="air_pressure_at_cloud_top"/>
    <attribute name="long_name" type="String" value="air_pressure_at_cloud_top"/>
    <attribute name="units" type="String" value="Pa"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudTopPres_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CloudTopPres_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudTopPres_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="RelHumid_D">
    <attribute name="standard_name" type="String" value="relative_humidity"/>
    <attribute name="long_name" type="String" value="relative_humidity"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="RelHumid_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="RelHumid_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="H2OVapMMR_D">
    <attribute name="standard_name" type="String" value="humidity_mixing_ratio"/>
    <attribute name="long_name" type="String" value="humidity_mixing_ratio"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="H2OVapMMR_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="H2OVapMMR_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="H2OVapMMR_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="Temperature_D">
    <attribute name="standard_name" type="String" value="air_temperature"/>
    <attribute name="long_name" type="String" value="air_temperature"/>
    <attribute name="units" type="String" value="K"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="Temperature_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="Temperature_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="Temperature_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="CO_total_column_D">
    <attribute name="standard_name" type="String" value="atmosphere_moles_of_carbon_monoxide"/>
    <attribute name="long_name" type="String" value="atmosphere_moles_of_carbon_monoxide"/>
    <attribute name="units" type="String" value="mol"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CO_total_column_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CO_total_column_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CH4_VMR_eff_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CH4_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CH4_VMR_eff_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CH4_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CH4_VMR_eff_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CH4_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CH4_VMR_eff_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CH4_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CO_VMR_eff_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CO_VMR_eff_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CO_VMR_eff_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CO_VMR_eff_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CO_dof_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CO_dof_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CO_dof_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CH4_dof_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CH4_dof_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CH4_dof_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CoarseCloudFrc_D">
    <attribute name="standard_name" type="String" value="cloud_area_fraction_in_atmosphere_layer"/>
    <attribute name="long_name" type="String" value="cloud_area_fraction_in_atmosphere_layer"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudFrc_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudFrc_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudTemp_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudTemp_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudTemp_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudPres_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudPres_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="CoarseCloudPres_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="FineCloudFrc_D">
    <attribute name="standard_name" type="String" value="cloud_area_fraction_in_atmosphere_layer"/>
    <attribute name="long_name" type="String" value="cloud_area_fraction_in_atmosphere_layer"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="FineCloudFrc_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="FineCloudFrc_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="CloudTopTemp_D">
    <attribute name="standard_name" type="String" value="air_temperature_at_cloud_top"/>
    <attribute name="long_name" type="String" value="air_temperature_at_cloud_top"/>
    <attribute name="units" type="String" value="K"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudTopTemp_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="CloudTopTemp_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CloudTopTemp_D_err">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropPres_D">
    <attribute name="standard_name" type="String" value="tropopause_air_pressure"/>
    <attribute name="long_name" type="String" value="tropopause_air_pressure"/>
    <attribute name="units" type="String" value="Pa"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropPres_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TropPres_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropHeight_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropHeight_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TropHeight_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropTemp_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TropTemp_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TropTemp_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="CO_Verticality_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CO_Verticality_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CO_Verticality_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="RelHumidLiq_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="RelHumidLiq_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="RelHumidLiq_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="H2OPrsLvls_D Latitude Longitude"/>
  </variable>
  <variable name="CO_Eff_Press_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CO_Eff_Press_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CO_Eff_Press_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CO_Layers_D Latitude Longitude"/>
  </variable>
  <variable name="CH4_Eff_Press_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="CH4_Eff_Press_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="CH4_Eff_Press_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="CoarseCldLyrs_D Latitude Longitude"/>
  </variable>
  <variable name="TotalCounts_MW_A">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotH2OVap_MW_A">
    <attribute name="standard_name" type="String" value="atmosphere_water_vapor_content"/>
    <attribute name="long_name" type="String" value="atmosphere_water_vapor_content"/>
    <attribute name="units" type="String" value="kg/m2"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotH2OVap_MW_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotH2OVap_MW_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="EmisMW_MW_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="MWEmisLvls_A Latitude Longitude"/>
  </variable>
  <variable name="EmisMW_MW_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="MWEmisLvls_A Latitude Longitude"/>
  </variable>
  <variable name="EmisMW_MW_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="MWEmisLvls_A Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_MW_A">
    <attribute name="standard_name" type="String" value="geopotential_height"/>
    <attribute name="long_name" type="String" value="geopotential_height"/>
    <attribute name="units" type="String" value="m"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A_MW Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_MW_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A_MW Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_MW_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A_MW Latitude Longitude"/>
  </variable>
  <variable name="Temperature_MW_A">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A_MW Latitude Longitude"/>
  </variable>
  <variable name="Temperature_MW_A_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A_MW Latitude Longitude"/>
  </variable>
  <variable name="Temperature_MW_A_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_A_MW Latitude Longitude"/>
  </variable>
  <variable name="TotalCounts_MW_D">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotH2OVap_MW_D">
    <attribute name="standard_name" type="String" value="atmosphere_water_vapor_content"/>
    <attribute name="long_name" type="String" value="atmosphere_water_vapor_content"/>
    <attribute name="units" type="String" value="kg/m2"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TotH2OVap_MW_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="TotH2OVap_MW_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="EmisMW_MW_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="MWEmisLvls_D Latitude Longitude"/>
  </variable>
  <variable name="EmisMW_MW_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="MWEmisLvls_D Latitude Longitude"/>
  </variable>
  <variable name="EmisMW_MW_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="MWEmisLvls_D Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_MW_D">
    <attribute name="standard_name" type="String" value="geopotential_height"/>
    <attribute name="long_name" type="String" value="geopotential_height"/>
    <attribute name="units" type="String" value="m"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D_MW Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_MW_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D_MW Latitude Longitude"/>
  </variable>
  <variable name="GPHeight_MW_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D_MW Latitude Longitude"/>
  </variable>
  <variable name="Temperature_MW_D">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D_MW Latitude Longitude"/>
  </variable>
  <variable name="Temperature_MW_D_ct">
    <attribute name="missing_value" type="int" value="0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D_MW Latitude Longitude"/>
  </variable>
  <variable name="Temperature_MW_D_sdev">
    <attribute name="missing_value" type="float" value="-9999.0"/>
    <attribute name="coordinates" type="String" value="TempPrsLvls_D_MW Latitude Longitude"/>
  </variable>
  <variable name="Latitude">
    <attribute name="standard_name" type="String" value="latitude"/>
    <attribute name="long_name" type="String" value="latitude"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="Longitude">
    <attribute name="standard_name" type="String" value="longitude"/>
    <attribute name="long_name" type="String" value="longitude"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="LandSeaMask">
    <attribute name="standard_name" type="String" value="land_binary_mask"/>
    <attribute name="long_name" type="String" value="land_binary_mask"/>
    <attribute name="missing_value" type="int" value="0"/>
  </variable>
  <variable name="Topography">
    <attribute name="standard_name" type="String" value="height_above_reference_ellipsoid"/>
    <attribute name="long_name" type="String" value="height_above_reference_ellipsoid"/>
    <attribute name="units" type="String" value="m"/>
    <attribute name="missing_value" type="float" value="-9999.0"/>
  </variable>
  <variable name="TempPrsLvls_A" shape="TempPrsLvls" type="float">
    <attribute name="standard_name" type="String" value="Pressure"/>
    <attribute name="long_name" type="String" value="Pressure levels temperature profile"/>
    <attribute name="units" type="String" value="hPa"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>1000.000000 925.000000 850.000000 700.000000 600.000000 500.000000 400.000000 300.000000 250.000000 200.000000 150.000000 100.000000 70.000000 50.000000 30.000000 20.000000 15.000000 10.000000 7.000000 5.000000 3.000000 2.000000 1.500000 1.000000 </values>
  </variable>
  <variable name="H2OPrsLvls_A" shape="H2OPrsLvls" type="float">
    <attribute name="standard_name" type="String" value="Pressure"/>
    <attribute name="long_name" type="String" value="Pressure layers water vapor profile"/>
    <attribute name="units" type="String" value="hPa"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>1000.000000 925.000000 850.000000 700.000000 600.000000 500.000000 400.000000 300.000000 250.000000 200.000000 150.000000 100.000000</values>
  </variable>
  <variable name="CoarseCldLyrs_A" shape="CoarseCldLyrs" type="int">
    <attribute name="standard_name" type="String" value="Altitude_indices"/>
    <attribute name="long_name" type="String" value="Altitude indices of Cloud Layer"/>
    <attribute name="comment" type="String" value="0-low, 1-middle, 2-high"/>
    <attribute name="positive" type="String" value="up"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>0 1 2</values>
  </variable>
  <variable name="CO_Layers_A" shape="CO_Layers" type="int">
    <attribute name="standard_name" type="String" value="Pressure_support_indices"/>
    <attribute name="long_name" type="String" value="Pressure support indices"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>45 56 63 70 81 89 93</values>
  </variable>
  <variable name="CH4_Layers_A" shape="CH4_Layers" type="int">
    <attribute name="standard_name" type="String" value="Pressure_support_indices"/>
    <attribute name="long_name" type="String" value="Pressure support indices"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>51 60 67</values>
  </variable>
  <variable name="IREmisLvls_A" shape="EmisLvls" type="float">
    <attribute name="standard_name" type="String" value="Frequency"/>
    <attribute name="long_name" type="String" value="Frequency of reported IR emissivity"/>
    <attribute name="units" type="String" value="1/cm"/>
    <attribute name="positive" type="String" value="up"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>832.000000 961.000000 1203.000000 2616.000000</values>
  </variable>
  <variable name="TempPrsLvls_D" shape="TempPrsLvls_1" type="float">
    <attribute name="standard_name" type="String" value="Pressure"/>
    <attribute name="long_name" type="String" value="Pressure levels temperature profile"/>
    <attribute name="units" type="String" value="hPa"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>1000.000000 925.000000 850.000000 700.000000 600.000000 500.000000 400.000000 300.000000 250.000000 200.000000 150.000000 100.000000 70.000000 50.000000 30.000000 20.000000 15.000000 10.000000 7.000000 5.000000 3.000000 2.000000 1.500000 1.000000 </values>
  </variable>
  <variable name="H2OPrsLvls_D" shape="H2OPrsLvls_1" type="float">
    <attribute name="standard_name" type="String" value="Pressure"/>
    <attribute name="long_name" type="String" value="Pressure layers water vapor profile"/>
    <attribute name="units" type="String" value="hPa"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>1000.000000 925.000000 850.000000 700.000000 600.000000 500.000000 400.000000 300.000000 250.000000 200.000000 150.000000 100.000000</values>
  </variable>
  <variable name="CoarseCldLyrs_D" shape="CoarseCldLyrs_1" type="int">
    <attribute name="standard_name" type="String" value="Altitude_indices"/>
    <attribute name="long_name" type="String" value="Altitude indices of Cloud Layer"/>
    <attribute name="comment" type="String" value="0-low, 1-middle, 2-high"/>
    <attribute name="positive" type="String" value="up"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>0 1 2</values>
  </variable>
  <variable name="CO_Layers_D" shape="CO_Layers_1" type="int">
    <attribute name="standard_name" type="String" value="Pressure_support_indices"/>
    <attribute name="long_name" type="String" value="Pressure support indices"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>45 56 63 70 81 89 93</values>
  </variable>
  <variable name="CH4_Layers_D" shape="CH4_Layers_1" type="int">
    <attribute name="standard_name" type="String" value="Pressure_support_indices"/>
    <attribute name="long_name" type="String" value="Pressure support indices"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>51 60 67</values>
  </variable>
  <variable name="IREmisLvls_D" shape="EmisLvls_1" type="float">
    <attribute name="standard_name" type="String" value="Frequency"/>
    <attribute name="long_name" type="String" value="Frequency of reported IR emissivity"/>
    <attribute name="units" type="String" value="1/cm"/>
    <attribute name="positive" type="String" value="up"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>832.000000 961.000000 1203.000000 2616.000000</values>
  </variable>
  <variable name="TempPrsLvls_A_MW" shape="TempPrsLvls_2" type="float">
    <attribute name="standard_name" type="String" value="Pressure"/>
    <attribute name="long_name" type="String" value="Pressure levels temperature profile"/>
    <attribute name="units" type="String" value="hPa"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>1000.000000 925.000000 850.000000 700.000000 600.000000 500.000000 400.000000 300.000000 250.000000 200.000000 150.000000 100.000000 70.000000 50.000000 30.000000 20.000000 15.000000 10.000000 7.000000 5.000000 3.000000 2.000000 1.500000 1.000000 </values>
  </variable>
  <variable name="H2OPrsLvls_A_MW" shape="H2OPrsLvls_2" type="float">
    <attribute name="standard_name" type="String" value="Pressure"/>
    <attribute name="long_name" type="String" value="Pressure layers water vapor profile"/>
    <attribute name="units" type="String" value="hPa"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>1000.000000 925.000000 850.000000 700.000000 600.000000 500.000000 400.000000 300.000000 250.000000 200.000000 150.000000 100.000000</values>
  </variable>
  <variable name="CoarseCldLyrs_A_MW" shape="CoarseCldLyrs_2" type="int">
    <attribute name="standard_name" type="String" value="Altitude_indices"/>
    <attribute name="long_name" type="String" value="Altitude indices of Cloud Layer"/>
    <attribute name="comment" type="String" value="0-low, 1-middle, 2-high"/>
    <attribute name="positive" type="String" value="up"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>0 1 2</values>
  </variable>
  <variable name="CO_Layers_A_MW" shape="CO_Layers_2" type="int">
    <attribute name="standard_name" type="String" value="Pressure_support_indices"/>
    <attribute name="long_name" type="String" value="Pressure support indices"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>45 56 63 70 81 89 93</values>
  </variable>
  <variable name="CH4_Layers_A_MW" shape="CH4_Layers_2" type="int">
    <attribute name="standard_name" type="String" value="Pressure_support_indices"/>
    <attribute name="long_name" type="String" value="Pressure support indices"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>51 60 67</values>
  </variable>
  <variable name="MWEmisLvls_A" shape="MWEmisLvls_2" type="float">
    <attribute name="standard_name" type="String" value="Frequency"/>
    <attribute name="long_name" type="String" value="Frequency of reported MW emissivity"/>
    <attribute name="units" type="String" value="GHz"/>
    <attribute name="positive" type="String" value="up"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>23.799999 50.299999 89.000000</values>
  </variable>
  <variable name="TempPrsLvls_D_MW" shape="TempPrsLvls_3" type="float">
    <attribute name="standard_name" type="String" value="Pressure"/>
    <attribute name="long_name" type="String" value="Pressure levels temperature profile"/>
    <attribute name="units" type="String" value="hPa"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>1000.000000 925.000000 850.000000 700.000000 600.000000 500.000000 400.000000 300.000000 250.000000 200.000000 150.000000 100.000000 70.000000 50.000000 30.000000 20.000000 15.000000 10.000000 7.000000 5.000000 3.000000 2.000000 1.500000 1.000000 </values>
  </variable>
  <variable name="H2OPrsLvls_D_MW" shape="H2OPrsLvls_3" type="float">
    <attribute name="standard_name" type="String" value="Pressure"/>
    <attribute name="long_name" type="String" value="Pressure layers water vapor profile"/>
    <attribute name="units" type="String" value="hPa"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>1000.000000 925.000000 850.000000 700.000000 600.000000 500.000000 400.000000 300.000000 250.000000 200.000000 150.000000 100.000000</values>
  </variable>
  <variable name="CoarseCldLyrs_D_MW" shape="CoarseCldLyrs_3" type="int">
    <attribute name="standard_name" type="String" value="Altitude_indices"/>
    <attribute name="long_name" type="String" value="Altitude indices of Cloud Layer"/>
    <attribute name="comment" type="String" value="0-low, 1-middle, 2-high"/>
    <attribute name="positive" type="String" value="up"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>0 1 2</values>
  </variable>
  <variable name="CO_Layers_D_MW" shape="CO_Layers_3" type="int">
    <attribute name="standard_name" type="String" value="Pressure_support_indices"/>
    <attribute name="long_name" type="String" value="Pressure support indices"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>45 56 63 70 81 89 93</values>
  </variable>
  <variable name="CH4_Layers_D_MW" shape="CH4_Layers_3" type="int">
    <attribute name="standard_name" type="String" value="Pressure_support_indices"/>
    <attribute name="long_name" type="String" value="Pressure support indices"/>
    <attribute name="positive" type="String" value="down"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>51 60 67</values>
  </variable>
  <variable name="MWEmisLvls_D" shape="MWEmisLvls_3" type="float">
    <attribute name="standard_name" type="String" value="Frequency"/>
    <attribute name="long_name" type="String" value="Frequency of reported MW emissivity"/>
    <attribute name="units" type="String" value="GHz"/>
    <attribute name="positive" type="String" value="up"/>
    <attribute name="_CoordinateAxisType" type="String" value="GeoZ"/>
    <values>23.799999 50.299999 89.000000</values>
  </variable>
</netcdf>
'''
from lxml import etree
class NcmlReader(Processor):
    def parse(self):
        elem = self.parser.xml
        ncml = {'variables': []}

        ncml['identifier'] = elem.attrib.get('location', '')
        for variable in extract_elems(elem, ['variable']):
            v = {}
            v['name'] = variable.attrib.get('name', '')
            v['attributes'] = []
            for att in extract_elems(variable, ['attribute']):
                a = {}
                for key, value in att.attrib.iteritems():
                    tag = extract_element_tag(key)
                    if tag == 'values':
                        continue
                    
                    a[tag] = value.strip()
                    
                if a:
                    v['attributes'] += [a]
            
            v = tidy_dict(v)
            if v:
                ncml['variables'].append(v)

        return tidy_dict(ncml)

reader = NcmlReader({}, ncml, 'www.example.com', '')
reader.parse()

{'identifier': '/opendap/hyrax/Aqua_AIRS_Level3/AIRX3ST8.005/2007/AIRS.2007.01.02.L3.RetStd008.v5.0.14.0.G07219003002.hdf',
 'variables': [{'attributes': [{'name': 'missing_value',
     'type': 'int',
     'value': '0'}],
   'name': 'TotalCounts_A'},
  {'attributes': [{'name': 'standard_name',
     'type': 'String',
     'value': 'atmosphere_cloud_liquid_water_content'},
    {'name': 'long_name',
     'type': 'String',
     'value': 'atmosphere_cloud_liquid_water_content'},
    {'name': 'units', 'type': 'String', 'value': 'kg/m2'},
    {'name': 'missing_value', 'type': 'float', 'value': '-9999.0'}],
   'name': 'TotCldLiqH2O_A'},
  {'attributes': [{'name': 'missing_value', 'type': 'int', 'value': '0'}],
   'name': 'TotCldLiqH2O_A_ct'},
  {'attributes': [{'name': 'missing_value',
     'type': 'float',
     'value': '-9999.0'}],
   'name': 'TotCldLiqH2O_A_sdev'},
  {'attributes': [{'name': 'missing_value',
     'type': 'float',
     'value': '-9999.0'}],
   'name': 'TotCldLiqH2O_A_err'},
